In [21]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
# tqdm().pandas()
import re
import string
import matplotlib.pyplot as plt
%matplotlib inline

# import cv2
from os import listdir
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow_hub as hub  #pip install tensorflow_hub
import os
import tokenization 
from tqdm import tqdm_notebook
from keras import backend as K
# Initialize session
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
sess = tf.Session()
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) #command to run codeon multiple gpu



# Params for bert model and tokenization
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

from keras_lr_finder import LRFinder
import talos as ta
from pprint import pprint
import talos
tf.logging.set_verbosity(tf.logging.ERROR)
from random import choice
import gc
from talos.model.normalizers import lr_normalizer


Device mapping: no known devices.


In [22]:
np.random.seed(42)
tf.set_random_seed(42)

In [47]:
def get_df(file):
    return pd.read_csv(file,sep = '\t')
train_df = get_df('C:\\Users\\Dell\\Desktop\\ML\\archive\\splits\\train_ids.txt')
train_df


,1057393737372966912
0,1057149180148240384
1,1113184475658293251
2,1108665597510991873
3,1108732814202228737
4,1035921045734211585
...,...
134817,1114657084849758209
134818,1107892812102668288
134819,1105670090098380806
134820,1108078550391894017


In [46]:
test_df = get_df('C:\\Users\\Dell\\Desktop\\ML\\archive\\splits\\test_ids.txt')
test_df

,1117460975903334410
0,1043075096544509952
1,1115669983680454656
2,1035902082199572486
3,1036092005741256705
4,1037147555367280640
...,...
9994,1109972327897460736
9995,1106756324803731456
9996,1114945824851730434
9997,1105122471743635456


In [48]:
val_df = get_df('C:\\Users\\Dell\\Desktop\\ML\\archive\\splits\\val_ids.txt')
val_df

,1062433411107713025
0,1045809514740666370
1,1107854991774138368
2,1054203094546829313
3,1035252480215592966
4,1037047746593603584
...,...
4994,1114923019942514688
4995,1114673262427086849
4996,1113637700748419072
4997,1110005215003066380


In [58]:
import pandas as pd

# Load the JSON data
with open('archive\MMHS150K_GT.json', 'r') as f:
    data = json.load(f)

# Create a list to store the data
tweets_data = []

# Iterate through each tweet in the data and append it to the list
for tweet_id, tweet_info in data.items():
    tweets_data.append({
        'Tweet ID': tweet_id,
        'Image URL': tweet_info['img_url'],
        'Labels': tweet_info['labels'],
        'Tweet URL': tweet_info['tweet_url'],
        'Tweet Text': tweet_info['tweet_text'],
        'Labels_str': tweet_info['labels_str']
    })

# Create a DataFrame from the list
df = pd.DataFrame(tweets_data)

# Display the DataFrame
df.head(10)


,Tweet ID,Image URL,Labels,Tweet URL,Tweet Text,Labels_str
0,1114679353714016256,http://pbs.twimg.com/tweet_video_thumb/D3gi9MH...,"[4, 1, 3]",https://twitter.com/user/status/11146793537140...,@FriskDontMiss Nigga https://t.co/cAsaLWEpue,"[Religion, Racist, Homophobe]"
1,1063020048816660480,http://pbs.twimg.com/ext_tw_video_thumb/106301...,"[5, 5, 5]",https://twitter.com/user/status/10630200488166...,My horses are retarded https://t.co/HYhqc6d5WN,"[OtherHate, OtherHate, OtherHate]"
2,1108927368075374593,http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg,"[0, 0, 0]",https://twitter.com/user/status/11089273680753...,“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL S...,"[NotHate, NotHate, NotHate]"
3,1114558534635618305,http://pbs.twimg.com/ext_tw_video_thumb/111401...,"[1, 0, 0]",https://twitter.com/user/status/11145585346356...,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,"[Racist, NotHate, NotHate]"
4,1035252480215592966,http://pbs.twimg.com/media/Dl30pGIU8AAVGxO.jpg,"[1, 0, 1]",https://twitter.com/user/status/10352524802155...,“EVERYbody calling you Nigger now!” https://t....,"[Racist, NotHate, Racist]"
5,1106978219654303744,http://pbs.twimg.com/media/D1zG0qnX4AAw9SC.jpg,"[0, 0, 0]",https://twitter.com/user/status/11069782196543...,“ real ass bitch give a fuck boutta nigga” htt...,"[NotHate, NotHate, NotHate]"
6,1113920043568463874,http://pbs.twimg.com/media/D3VwYEKW4AYz4vk.jpg,"[5, 1, 1]",https://twitter.com/user/status/11139200435684...,@WhiteHouse @realDonaldTrump Fuck ice. White s...,"[OtherHate, Racist, Racist]"
7,1114588617693966336,http://pbs.twimg.com/media/D3fQcCCWAAIG8tO.jpg,"[0, 0, 0]",https://twitter.com/user/status/11145886176939...,Day’s a cunt https://t.co/Ie6QZReHsw,"[NotHate, NotHate, NotHate]"
8,1045809514740666370,http://pbs.twimg.com/media/DoN2KFmXcAAIT-Y.jpg,"[3, 3, 0]",https://twitter.com/user/status/10458095147406...,#sissy faggot https://t.co/bm1nk8HcYO,"[Homophobe, Homophobe, NotHate]"
9,1108178453910695936,http://pbs.twimg.com/ext_tw_video_thumb/110817...,"[4, 0, 3]",https://twitter.com/user/status/11081784539106...,@Gloriko_ Nigga what? https://t.co/nOwIJtgtU1,"[Religion, NotHate, Homophobe]"


In [59]:
test_df.shape


(9999, 1)

In [60]:
train_df.shape

(134822, 1)

In [62]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Now you can use the tokenizer to tokenize text


In [4]:
from tokenization import FullTokenizer

class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.
  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    bert_module =  hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    vocab_file, do_lower_case = sess.run(
        [
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"],
        ]
    )

    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

def convert_single_example(tokenizer, example, max_seq_length=256):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        input_ids = [0] * max_seq_length
        input_mask = [0] * max_seq_length
        segment_ids = [0] * max_seq_length
        label = 0
        return input_ids, input_mask, segment_ids, label

    tokens_a = tokenizer.tokenize(example.text_a)
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0 : (max_seq_length - 2)]

    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    return input_ids, input_mask, segment_ids, example.label

def convert_examples_to_features(tokenizer, examples, max_seq_length=256):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    input_ids, input_masks, segment_ids, labels = [], [], [], []
    for example in tqdm_notebook(examples, desc="Converting examples to features"):
        input_id, input_mask, segment_id, label = convert_single_example(
            tokenizer, example, max_seq_length
        )
        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        labels.append(label)
    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(segment_ids),
        np.array(labels).reshape(-1, 1),
    )

def convert_text_to_examples(texts, labels):
    """Create InputExamples"""
    InputExamples = []
    for text, label in zip(texts, labels):
        InputExamples.append(
            InputExample(guid=None, text_a=" ".join(text), text_b=None, label=label)
        )
    return InputExamples

# Instantiate tokenizer
tokenizer = create_tokenizer_from_hub_module()

# Convert data to InputExample format
train_examples = convert_text_to_examples(train_text, trainY)
test_examples = convert_text_to_examples(test_text, testY)

# Convert to features
(train_input_ids, train_input_masks, train_segment_ids, trainY 
) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)
(test_input_ids, test_input_masks, test_segment_ids, testY
) = convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

ImportError: cannot import name 'FullTokenizer' from 'tokenization' (c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\tokenization\__init__.py)